In [56]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn import tree, metrics
DATA_PATH = '/home/ellen/data'

In [57]:
data = pd.read_excel(f"{DATA_PATH}/yahoo_data.xlsx")
data.sample(5)

,Date,Open,High,Low,Close*,Adj Close**,Volume
434,"Aug 06, 2021",35077.44,35246.79,35077.44,35208.51,35208.51,231570000
1188,"Aug 08, 2018",25615.72,25634.11,25557.48,25583.75,25583.75,217770000
1207,"Jul 12, 2018",24802.90,24939.97,24802.90,24924.89,24924.89,233150000
187,"Aug 01, 2022",32755.71,32972.03,32640.79,32798.40,32798.40,321180000
140,"Oct 06, 2022",30206.13,30329.61,29859.34,29926.94,29926.94,306840000


In [58]:
data['Inside Day'] = (data['High'] < data['High'].shift(1)) & (data['Low'] > data['Low'].shift(1))

In [59]:
data.sample(5)

,Date,Open,High,Low,Close*,Adj Close**,Volume,Inside Day
770,"Apr 07, 2020",23537.44,23617.24,22634.45,22653.86,22653.86,594660000,False
370,"Nov 05, 2021",36268.75,36484.75,36190.20,36327.95,36327.95,344600000,False
14,"Apr 10, 2023",33425.25,33590.24,33343.43,33586.52,33586.52,232920000,False
1241,"May 23, 2018",24757.71,24889.46,24667.12,24886.81,24886.81,399610000,False
96,"Dec 08, 2022",33695.97,33899.17,33642.46,33781.48,33781.48,271830000,False


In [60]:
data['Inside Day Final'] = data['Inside Day'].replace({True: 1, False : 0})
data[['Inside Day', 'Inside Day Final']].sample(10)

/tmp/ipykernel_1545/4013368310.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['Inside Day Final'] = data['Inside Day'].replace({True: 1, False : 0})


,Inside Day,Inside Day Final
490,False,0
1243,False,0
409,True,1
751,False,0
515,False,0
989,True,1
274,True,1
11,False,0
602,True,1
714,False,0


In [61]:
features = data[['Open', 'High', 'Low', 'Close*', 'Adj Close**', 'Volume']]
target = data["Inside Day Final"]

In [62]:
from sklearn.model_selection import StratifiedShuffleSplit

# Get the split indexes
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)
train_index, test_index = next(sss.split(features, target))

#write code here
features_train = features.iloc[train_index]
target_train =target.iloc[train_index]

features_test = features.iloc[test_index]
target_test = target.iloc[test_index]

In [63]:
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression(penalty='l2', C=1.0, random_state=42, max_iter=1000, class_weight='balanced')
LR = LR.fit(features_train, target_train)
target_predict=LR.predict(features_test)

In [64]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy_valueR = accuracy_score(target_test, target_predict)
print(f"Accuracy of regularized model: {accuracy_valueR}")

precision_valueR = precision_score(target_test, target_predict, average = 'weighted')
print(f"Precision of regularized model: {precision_valueR}")

recall_valueR = recall_score(target_test, target_predict, average = 'weighted')
print(f"Recall of regularized model: {recall_valueR}")

fscore_valueR = f1_score(target_test, target_predict, average = 'weighted')
print(f"F-Score of regularized model: {fscore_valueR}")

Accuracy of regularized model: 0.5105820105820106
Precision of regularized model: 0.863178077066966
Recall of regularized model: 0.5105820105820106
F-Score of regularized model: 0.6055064246990982


In [65]:
conf_matrix = confusion_matrix(target_test,target_predict)
print(conf_matrix)

[[169 174]
 [ 11  24]]


In [66]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [67]:
from sklearn import model_selection
features_train, features_test, target_train, target_test = model_selection.train_test_split(features, target, test_size=0.25)

# Train the model
from sklearn.ensemble import RandomForestClassifier

random_forest_clf = RandomForestClassifier(n_estimators=10, 
                             max_depth=None, 
                             min_samples_split=2, 
                             random_state=0)

random_forest_clf = random_forest_clf.fit(features_train, target_train)

random_forest_test_predictions = random_forest_clf.predict(features_test)

In [36]:
conf_matrix = confusion_matrix(target_test,random_forest_test_predictions)
print(conf_matrix)

[[281   5]
 [ 29   0]]


In [37]:
print("Accuracy:", metrics.accuracy_score(target_test, random_forest_test_predictions))
print(metrics.classification_report(target_test, random_forest_test_predictions))

Accuracy: 0.8920634920634921
              precision    recall  f1-score   support

           0       0.91      0.98      0.94       286
           1       0.00      0.00      0.00        29

    accuracy                           0.89       315
   macro avg       0.45      0.49      0.47       315
weighted avg       0.82      0.89      0.86       315



In [38]:
d_tree_model = tree.DecisionTreeClassifier()  
d_tree_model = d_tree_model.fit(features_train, target_train)

In [39]:
test_predictions = d_tree_model.predict(features_test)
print(test_predictions[0:5])

[0 0 0 0 0]


In [40]:
print("Accuracy:", metrics.accuracy_score(target_test, test_predictions))

Accuracy: 0.8126984126984127


In [44]:
#Write code
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy_valueR = accuracy_score(target_test, test_predictions)
print(f"Accuracy of regularized model: {accuracy_valueR}")

precision_valueR = precision_score(target_test, test_predictions, average = 'weighted')
print(f"Precision of regularized model: {precision_valueR}")

recall_valueR = recall_score(target_test, test_predictions, average = 'weighted')
print(f"Recall of regularized model: {recall_valueR}")

fscore_valueR = f1_score(target_test, test_predictions, average = 'weighted')
print(f"F-Score of regularized model: {fscore_valueR}")

Accuracy of regularized model: 0.8126984126984127
Precision of regularized model: 0.826112556529136
Recall of regularized model: 0.8126984126984127
F-Score of regularized model: 0.8193051706279219


In [68]:
conf_matrix = confusion_matrix(target_test,test_predictions)
print(conf_matrix)

[[259  30]
 [ 22   4]]


In [69]:
print(metrics.classification_report(target_test, test_predictions))

              precision    recall  f1-score   support

           0       0.92      0.90      0.91       289
           1       0.12      0.15      0.13        26

    accuracy                           0.83       315
   macro avg       0.52      0.53      0.52       315
weighted avg       0.86      0.83      0.84       315



In [70]:
from sklearn import svm

svm_classifier = svm.SVC()
svm_classifier = svm_classifier.fit(features_train, target_train)  
svm_test_predictions = svm_classifier.predict(features_test)
print("SVM Accuracy:", metrics.accuracy_score(target_test, svm_test_predictions))

SVM Accuracy: 0.9174603174603174


In [71]:
conf_matrix = confusion_matrix(target_test,svm_test_predictions)
print(conf_matrix)

[[289   0]
 [ 26   0]]


In [72]:
print(metrics.classification_report(target_test, svm_test_predictions))

              precision    recall  f1-score   support

           0       0.92      1.00      0.96       289
           1       0.00      0.00      0.00        26

    accuracy                           0.92       315
   macro avg       0.46      0.50      0.48       315
weighted avg       0.84      0.92      0.88       315



/home/ellen/jvenv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/ellen/jvenv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/ellen/jvenv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


In [73]:
from sklearn.neighbors import KNeighborsClassifier

knn_clf = KNeighborsClassifier()
knn_clf = knn_clf.fit(features_train, target_train)
knn_test_predictions = knn_clf.predict(features_test)
print("KNN Accuracy:", metrics.accuracy_score(target_test, knn_test_predictions))

KNN Accuracy: 0.9142857142857143


In [74]:
conf_matrix = confusion_matrix(target_test,knn_test_predictions)
print(conf_matrix)

[[287   2]
 [ 25   1]]


In [75]:
print(metrics.classification_report(target_test, svm_test_predictions))

              precision    recall  f1-score   support

           0       0.92      1.00      0.96       289
           1       0.00      0.00      0.00        26

    accuracy                           0.92       315
   macro avg       0.46      0.50      0.48       315
weighted avg       0.84      0.92      0.88       315



/home/ellen/jvenv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/ellen/jvenv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/ellen/jvenv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
